# **IDL Assignment 6 - More Realistic Language Modeling & Recurrent Neural Networks**

## **Assigning Tensorflow version and importing the libraries required for the tasks**


In [ ]:
%tensorflow_version 2.x

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os

tf.__version__

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


'2.2.0'

In [ ]:
os.getcwd()
os.chdir("/content/drive/My Drive/Colab Notebooks/IDL /IDL Assignments/Assignment helper files") 


## **Fixed Length Sequenes**

### **Preprocess the text data** 

In [ ]:
!python prepare_data.py shakespeare_input.txt skp

2020-05-29 11:00:47.117101: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Split input into 22981 sequences...
Serialized 100 sequences...
Serialized 200 sequences...
Serialized 300 sequences...
Serialized 400 sequences...
Serialized 500 sequences...
Serialized 600 sequences...
Serialized 700 sequences...
Serialized 800 sequences...
Serialized 900 sequences...
Serialized 1000 sequences...
Serialized 1100 sequences...
Serialized 1200 sequences...
Serialized 1300 sequences...
Serialized 1400 sequences...
Serialized 1500 sequences...
Serialized 1600 sequences...
Serialized 1700 sequences...
Serialized 1800 sequences...
Serialized 1900 sequences...
Serialized 2000 sequences...
Serialized 2100 sequences...
Serialized 2200 sequences...
Serialized 2300 sequences...
Serialized 2400 sequences...
Serialized 2500 sequences...
Serialized 2600 sequences...
Serialized 2700 sequences...
Serialized 2800 sequences...
Serialized 2900

**total serialized seq is 22981**

**Loading the data from skp.tfrecords and skp_vocab**

The files mentioned are the output obtained after running the program *prepare_data.py* for the Shakespeare data. These files are loaded as data using tf.data and create a vocabulary dictionary 

**Note:** The vocab contains elements as dict with (key,val) as (character, index). Reverse mapping is done and stored as ind_to_ch which has (key,val) as (index,character)

In [ ]:
from prepare_data import parse_seq
import pickle

# this is just a datasets of "bytes" (not understandable)
data = tf.data.TFRecordDataset("skp.tfrecords")

# this maps a parser function that properly interprets the bytes over the dataset
# (with fixed sequence length 200)
# if you change the sequence length in preprocessing you also need to change it here
data = data.map(lambda x: parse_seq(x, 200))

# a map from characters to indices
vocab = pickle.load(open("skp_vocab", mode="rb"))
vocab_size = len(vocab)
# inverse mapping: indices to characters
ind_to_ch = {ind: ch for (ch, ind) in vocab.items()}

print(vocab)
print(vocab_size)

print("Indices to char")
print(ind_to_ch)

{'p': 1, 'C': 2, 'f': 3, 'w': 4, 'h': 5, ']': 6, 't': 7, '\n': 8, 'j': 9, 'e': 10, 'B': 11, 'o': 12, 'd': 13, "'": 14, '!': 15, 'O': 16, 'H': 17, 'U': 18, 'P': 19, 'M': 20, 'T': 21, '.': 22, '?': 23, '&': 24, 'Z': 25, 'A': 26, 'X': 27, ' ': 28, 'W': 29, 'q': 30, 'l': 31, 'J': 32, 'b': 33, '$': 34, 'D': 35, 'E': 36, 'F': 37, 'a': 38, '-': 39, ':': 40, 'I': 41, 'G': 42, 'c': 43, 'N': 44, 'Q': 45, 'V': 46, 'm': 47, ',': 48, 'L': 49, 'n': 50, 'R': 51, 'K': 52, ';': 53, 'g': 54, 'r': 55, 's': 56, 'Y': 57, 'x': 58, 'v': 59, 'S': 60, 'z': 61, 'u': 62, 'k': 63, 'i': 64, '3': 65, '[': 66, 'y': 67, '<S>': 0}
68
Indices to char
{1: 'p', 2: 'C', 3: 'f', 4: 'w', 5: 'h', 6: ']', 7: 't', 8: '\n', 9: 'j', 10: 'e', 11: 'B', 12: 'o', 13: 'd', 14: "'", 15: '!', 16: 'O', 17: 'H', 18: 'U', 19: 'P', 20: 'M', 21: 'T', 22: '.', 23: '?', 24: '&', 25: 'Z', 26: 'A', 27: 'X', 28: ' ', 29: 'W', 30: 'q', 31: 'l', 32: 'J', 33: 'b', 34: '$', 35: 'D', 36: 'E', 37: 'F', 38: 'a', 39: '-', 40: ':', 41: 'I', 42: 'G', 43: 

In [ ]:
data

<MapDataset shapes: (200,), types: tf.int32>

In [ ]:
def create_input_target(data):
    input_data = data[:-1]
    target_data = data[1:]
    return input_data, target_data


data = data.map(create_input_target)

In [ ]:
for x in data.take(1):
  print(tf.shape(x))
  print(type(x))
  print(repr(tf.shape(x)))
  print(x)

tf.Tensor([  2 199], shape=(2,), dtype=int32)
<class 'tuple'>
<tf.Tensor: shape=(2,), dtype=int32, numpy=array([  2, 199], dtype=int32)>
(<tf.Tensor: shape=(199,), dtype=int32, numpy=
array([ 0, 37, 64, 55, 56,  7, 28,  2, 64,  7, 64, 61, 10, 50, 40,  8, 11,
       10,  3, 12, 55, 10, 28,  4, 10, 28,  1, 55, 12, 43, 10, 10, 13, 28,
       38, 50, 67, 28,  3, 62, 55,  7,  5, 10, 55, 48, 28,  5, 10, 38, 55,
       28, 47, 10, 28, 56,  1, 10, 38, 63, 22,  8,  8, 26, 31, 31, 40,  8,
       60,  1, 10, 38, 63, 48, 28, 56,  1, 10, 38, 63, 22,  8,  8, 37, 64,
       55, 56,  7, 28,  2, 64,  7, 64, 61, 10, 50, 40,  8, 57, 12, 62, 28,
       38, 55, 10, 28, 38, 31, 31, 28, 55, 10, 56, 12, 31, 59, 10, 13, 28,
       55, 38,  7,  5, 10, 55, 28,  7, 12, 28, 13, 64, 10, 28,  7,  5, 38,
       50, 28,  7, 12, 28,  3, 38, 47, 64, 56,  5, 23,  8,  8, 26, 31, 31,
       40,  8, 51, 10, 56, 12, 31, 59, 10, 13, 22, 28, 55, 10, 56, 12, 31,
       59, 10, 13, 22,  8,  8, 37, 64, 55, 56,  7, 28,  2, 64,  7,

### **Batching data**

In [ ]:
## Declare the sizes of batch, shuffle and repeat

SHUFFLE_SIZE = 1000
BATCH_SIZE = 128
REPEAT_TIMES = 20

In [ ]:
def batch_shuffle_repeat(data):


    data = data.shuffle(SHUFFLE_SIZE)
    data = data.padded_batch(BATCH_SIZE, padded_shapes=None,drop_remainder=True)   
    # data = data.repeat(REPEAT_TIMES)


    return data

In [ ]:
dataset = batch_shuffle_repeat(data)


In [ ]:
for x in dataset.take(1):
  print(tf.shape(x)[0])
  print(tf.shape(x)[1])

  print(tf.shape(x)[2])

  print(type(x))
  print(repr(tf.shape(x)[2]))
  print(x)

tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(128, shape=(), dtype=int32)
tf.Tensor(199, shape=(), dtype=int32)
<class 'tuple'>
<tf.Tensor: shape=(), dtype=int32, numpy=199>
(<tf.Tensor: shape=(128, 199), dtype=int32, numpy=
array([[ 0, 12, 13, ..., 13, 28, 33],
       [ 0, 10, 55, ..., 28, 38,  7],
       [ 0, 28,  2, ..., 56, 62, 55],
       ...,
       [ 0, 13, 10, ..., 28,  1, 38],
       [ 0,  7,  5, ..., 13, 28, 54],
       [ 0, 50, 54, ..., 26, 56, 28]], dtype=int32)>, <tf.Tensor: shape=(128, 199), dtype=int32, numpy=
array([[12, 13, 67, ..., 28, 33, 10],
       [10, 55, 28, ..., 38,  7, 28],
       [28,  2, 12, ..., 62, 55, 10],
       ...,
       [13, 10, 28, ...,  1, 38, 64],
       [ 7,  5, 10, ..., 28, 54, 12],
       [50, 54, 28, ..., 56, 28,  5]], dtype=int32)>)


### **Model creation using Keras and printing text**

In [ ]:
# The embedding dimension
embedding_dim = 199

# Number of RNN units
rnn_units = 512

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, BATCH_SIZE)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 199)          13532     
_________________________________________________________________
gru (GRU)                    (128, None, 512)          1095168   
_________________________________________________________________
dense (Dense)                (128, None, 68)           34884     
Total params: 1,143,584
Trainable params: 1,143,584
Non-trainable params: 0
_________________________________________________________________


In [ ]:
opt = tf.optimizers.Adam()
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
test_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()

In [ ]:
# stereotypical train-step-with-function-annotation

@tf.function
def train_step(input_batch, target_batch):
    with tf.GradientTape(persistent=True) as tape:
        logits = model(input_batch,training=True)
        loss = loss_fn(target_batch, logits)
        # print(loss)

    varis = model.trainable_variables
    grads = tape.gradient(loss, varis)
    opt.apply_gradients(zip(grads, varis))

    return loss, logits

In [ ]:
import time
start = time.time()

steps = 0

epochs = 20
for epoch in range(epochs):
  print("="*90)
  print('Start of epoch %d' % (epoch+1,))
  
  batch_nr = 0

  for input_batch,target_batch in dataset:
      # steps = steps+1
      batch_start = time.time()

      loss, logits = train_step(input_batch, target_batch)

      # if not steps % 30:


      batch_nr = batch_nr+1
      batch_stop = time.time()
  #      rnn_try(batch_data)
      # train_acc_metric(target_batch, logits)
      # acc = train_acc_metric.result()
      print("Epoch: {} Batch Number: {} Loss: {} Time taken: {}".format(epoch+1,batch_nr,loss,batch_stop-batch_start))

      # print("Loss: {} Accuracy: {}".format(loss, acc))
      # train_acc_metric.reset_states()


stop = time.time()
print("took {} seconds\n".format(stop-start))
#start = time.time()


Start of epoch 1
Epoch: 1 Batch Number: 1 Loss: 4.219892501831055 Time taken: 3.700148582458496
Epoch: 1 Batch Number: 2 Loss: 4.197140693664551 Time taken: 0.21980023384094238
Epoch: 1 Batch Number: 3 Loss: 4.168467044830322 Time taken: 0.20259809494018555
Epoch: 1 Batch Number: 4 Loss: 4.126499652862549 Time taken: 0.21307063102722168
Epoch: 1 Batch Number: 5 Loss: 4.038734436035156 Time taken: 0.200700044631958
Epoch: 1 Batch Number: 6 Loss: 3.822652578353882 Time taken: 0.2007887363433838
Epoch: 1 Batch Number: 7 Loss: 4.333465576171875 Time taken: 0.20063257217407227
Epoch: 1 Batch Number: 8 Loss: 3.560145616531372 Time taken: 0.2050924301147461
Epoch: 1 Batch Number: 9 Loss: 3.7140934467315674 Time taken: 0.2062973976135254
Epoch: 1 Batch Number: 10 Loss: 3.753002166748047 Time taken: 0.20309877395629883
Epoch: 1 Batch Number: 11 Loss: 3.758662462234497 Time taken: 0.2037043571472168
Epoch: 1 Batch Number: 12 Loss: 3.734156608581543 Time taken: 0.19968175888061523
Epoch: 1 Batch 

In [ ]:
model_1 = build_model(vocab_size, embedding_dim, rnn_units,1)
model_1.set_weights(model.get_weights())

#### **Printing Text**

In [ ]:
num_generate = 2500


text_generated = []
#temp = 1.5
#h_t = tf.zeros([1,n_h])
start_char = '<S>'
choice = []
val = vocab[start_char]
choice.append(val)

model_1.reset_states()
for i in range(num_generate):
  # x_t = tf.one_hot(choice[-1:],depth = vocab_size)
  # a = (tf.matmul(x_t,w_xh))+ (tf.matmul(h_t, w_hh)) + b_h
  #print(type(a))
  # h_t = tf.nn.tanh(a)
  # logits = (tf.matmul(h_t, w_ho)) + b_o
  #print(logits)
  input_eval = choice[-1:]
  input_eval = tf.expand_dims(input_eval, 0)
  logits = model_1(input_eval)

  predictions = tf.nn.softmax(logits)
  #predictions = predictions[0,:]
  predictions = tf.squeeze(predictions, 0)
  #print("----------------------"*3)
  #print(predictions.shape)
  #print(predictions)
  #print(predictions.numpy().shape)
  #predictions = predictions / temp

  ## The predictions is a tensor 
  predictions = predictions.numpy()[0]
  #predictions = np.array(predictions)
  # print(predictions.shape)
  #print(predictions)
  char_choice = np.random.choice(vocab_size, p = predictions)
  #print(char_choice)
  choice.append(char_choice)
  #print(choice)
  #predictions = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
  #print(predictions)
  start_char = ind_to_ch[char_choice]
  #print(start_char)
  #print(input_eval)
  text_generated.append(start_char)
  
print("".join(text_generated))
print("="*90)


eire wickedfully have eater thy
reason, nor to our grace, Begum and mind.
Hours, because they pall with; his eyes behaved,
Than the heavens tell them beaten. This down
I, instigations of commosterous carp,
May sit in preservation come;
But, I'll strike the blooded spirit, or content
one envy: for thou shall you not feed of mon;
And throy it but to le, boy; and, if he
that false of alamotter is well manish'd in me,
The valiant vanous purposes but in she touch the devilier.

Clown:
My lord decessing of the sun than we have wop,
For you to grant him if thou desist your eyes.
Hostess our fathers above
France, if you do it.

VIOLA:
Hang thee, niece?

SLENDER:
O misery! I trust;' to offer him;
For we of languing, to blessed
So strong me in time Harry, let how her idle
issue under both apparel might consomation.
Fair soul is to better bear his name,
And stay too would conclusy,
Whose rascal confairing to by poor king,
As we are, though in this hat as my bed, and this
I will'd Charles; if you 

### **Model creation custom made and printing text**

In [ ]:
n_h = 512

## w_xh is input to hidden weight --> known as U from the literature
## w_hh is hidden to hidden weights --> known as W from the literature
## w_ho is hidden to output weights --> known as V from the literature
## b_h and b_o are the biases at the hidden layer and output layer


w_xh = tf.Variable(tf.initializers.glorot_uniform()([vocab_size,n_h]))

w_hh = tf.Variable(tf.initializers.glorot_uniform()([n_h,n_h]))
b_h = tf.Variable(tf.zeros([n_h]))

w_ho = tf.Variable(tf.initializers.glorot_uniform()([n_h,vocab_size]))
b_o = tf.Variable(tf.zeros([vocab_size]))

variables = [w_xh,w_hh,b_h,w_ho,b_o]

In [ ]:
opt = tf.optimizers.Adam()
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


@tf.function
def rnn_sequence(batch_data):
    with tf.GradientTape() as tape:
        h_t = tf.zeros([tf.shape(batch_data)[0],n_h])
        loss = tf.TensorArray(tf.float32,size=tf.shape(batch_data)[1]-1)

        for timestep in tf.range(tf.shape(batch_data)[1]-1):
            x_t = tf.one_hot(batch_data[:,timestep],vocab_size)
            h_t = tf.nn.tanh(tf.matmul(x_t,w_xh) + tf.matmul(h_t,w_hh) + b_h)
            logits = tf.matmul(h_t,w_ho) + b_o

            local_loss = loss_fn(batch_data[:,timestep+1],logits)

            loss = loss.write(timestep, local_loss)
        loss = loss.stack()

        batch_loss = tf.reduce_mean(loss)
        
    
    grads = tape.gradient(batch_loss, variables)
    opt.apply_gradients(zip(grads, variables))

    return batch_loss,h_t


In [ ]:
##### DONT RUN THIS BLOCK .. THIS IS JUST FOR TRIAL PURPOSES


###############################################################################################################################################
opt = tf.optimizers.Adam()
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

"""
        tf.print(tf.shape(batch_data))
        tf.print(type(batch_data))
        tf.print(repr(tf.shape(batch_data)))
        tf.print(batch_data)

        for i in tf.range(tf.shape(batch_data)[1]-1):
          tf.print(i)
"""



@tf.function
def rnn_try(batch_data):
    with tf.GradientTape() as tape:
        h_t = tf.zeros([tf.shape(batch_data)[0],n_h])
        loss = tf.TensorArray(tf.float32,size=tf.shape(batch_data)[1]-1)
        

        for timestep in tf.range(tf.shape(batch_data)[1]-1):
            tf.print("="*100)
            tf.print(timestep)

            x_t = tf.one_hot(batch_data[:,timestep],vocab_size)
            tf.print(tf.shape(x_t))
            tf.print(type(x_t))
            tf.print(repr(tf.shape(x_t)))



            h_t = tf.nn.tanh(tf.matmul(x_t,w_xh) + tf.matmul(h_t,w_hh) + b_h)
            tf.print(tf.shape(h_t))
            tf.print(type(h_t))
            tf.print(repr(tf.shape(h_t)))


            logits = tf.matmul(h_t,w_ho) + b_o
            tf.print(tf.shape(logits))
            tf.print(type(logits))
            tf.print(repr(tf.shape(logits)))


            local_loss = loss_fn(batch_data[:,timestep+1],logits)
            tf.print(tf.shape(local_loss))
            tf.print(type(local_loss))
            tf.print(repr(tf.shape(local_loss)))


            loss = loss.write(timestep, local_loss)

        loss = loss.stack()
        tf.print(tf.shape(loss))
        tf.print(type(loss))
        tf.print(repr(tf.shape(loss)))





In [ ]:
import time

start = time.time()


batch_nr = 0
for batch_data in dataset:
      batch_start = time.time()
      batch_nr = batch_nr+1
      batch_loss,h_t = rnn_sequence(batch_data)
      batch_stop = time.time()
#      rnn_try(batch_data)
      print("Batch Number: {} Loss: {} Time taken: {}".format(batch_nr,batch_loss,batch_stop-batch_start))
#      if not steps % 100:
#          train_acc_metric(lbl_batch, logits)
#          acc = train_acc_metric.result()
#          print("Loss: {} Accuracy: {}".format(loss, acc))
#          train_acc_metric.reset_states()

stop = time.time()
print("took {} seconds\n".format(stop-start))

Batch Number: 1 Loss: 4.221291542053223 Time taken: 3.126525640487671
Batch Number: 2 Loss: 3.833584785461426 Time taken: 0.38123464584350586
Batch Number: 3 Loss: 3.5180790424346924 Time taken: 0.3758275508880615
Batch Number: 4 Loss: 3.583409309387207 Time taken: 0.3216116428375244
Batch Number: 5 Loss: 3.39400315284729 Time taken: 0.2955586910247803
Batch Number: 6 Loss: 3.4939541816711426 Time taken: 0.2938830852508545
Batch Number: 7 Loss: 3.3784704208374023 Time taken: 0.3131701946258545
Batch Number: 8 Loss: 3.363752603530884 Time taken: 0.2981090545654297
Batch Number: 9 Loss: 3.385077953338623 Time taken: 0.3008840084075928
Batch Number: 10 Loss: 3.3499796390533447 Time taken: 0.2968623638153076
Batch Number: 11 Loss: 3.3489768505096436 Time taken: 0.2958827018737793
Batch Number: 12 Loss: 3.3338100910186768 Time taken: 0.3241133689880371
Batch Number: 13 Loss: 3.311328887939453 Time taken: 0.3471994400024414
Batch Number: 14 Loss: 3.3387887477874756 Time taken: 0.348881244659

In [ ]:
characters = [0,5,6]
print(characters)

print(characters[-1:])

[0, 5, 6]
[6]


#### **Using np.random.choice**

In [ ]:
num_generate = 2500


text_generated = []
#temp = 1.5
#h_t = tf.zeros([1,n_h])
start_char = '<S>'
choice = []
val = vocab[start_char]
choice.append(val)

for i in range(num_generate):
  x_t = tf.one_hot(choice[-1:],depth = vocab_size)
  a = (tf.matmul(x_t,w_xh))+ (tf.matmul(h_t, w_hh)) + b_h
  #print(type(a))
  h_t = tf.nn.tanh(a)
  logits = (tf.matmul(h_t, w_ho)) + b_o
  #print(logits)
  predictions = tf.nn.softmax(logits)
  #predictions = predictions[0,:]
  # predictions = tf.squeeze(predictions, 0)
  #print("----------------------"*3)
  #print(predictions.shape)
  #print(predictions)
  #print(predictions.numpy().shape)
  #predictions = predictions / temp

  ## The predictions is a tensor 
  predictions = predictions.numpy()[0]
  #predictions = np.array(predictions)
  #print(predictions.shape)
  #print(predictions)
  char_choice = np.random.choice(vocab_size, p = predictions)
  #print(char_choice)
  choice.append(char_choice)
  #print(choice)
  #predictions = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
  #print(predictions)
  start_char = ind_to_ch[char_choice]
  #print(start_char)
  #print(input_eval)
  text_generated.append(start_char)
  
print("".join(text_generated))
print("="*90)


en. Who, ave adow a whole, the weam when yod a gear'l stand it sung, if it will so will dread desires, of is death
Is thou fell wear father,
But the grave inquest-to it again the baich's surper unto this honouraigs so swon
Next be the law and solered do is ne'er rearons and sworn, acch medie: I hear him: loviss wse decelseng my fous for unjmyed, they and oderry aftat of his no magoor tequit fount, thou now defey in his was one mather time should you known, would to pear thy grandat this gown, mansten Duke of seautance as thou art lie to:
I'll fable true to heart non misere gave of right
May your rejest and of it is He Isterm'd in sead my Pains?
Whise is thou not the devil truth. 'sid it adomen's own, hos thr foreless dead as how deverse make ary
overades of man, and you as
that thus ent his toss in grands 'tis im'ting and constate
Iven's as for mine in my
Francers.

KING JOAN:
If you? will be a sufford passion.
O hard for a seemirits, by this miscreed.

CARDINAL:
I am nature and sis br

#### **Using np.argmax**



In [ ]:
num_generate = 2500


text_generated = []
#temp = 1.5
h_t = tf.zeros([1,n_h])
start_char = '<S>'
choice = []
val = vocab[start_char]
choice.append(val)

for i in range(num_generate):
  x_t = tf.one_hot(choice[-1:],depth = vocab_size)
  a = (tf.matmul(x_t,w_xh))+ (tf.matmul(h_t, w_hh)) + b_h
  #print(type(a))
  h_t = tf.nn.tanh(a)
  logits = (tf.matmul(h_t, w_ho)) + b_o
  #print(logits)
  predictions = tf.nn.softmax(logits)
  #predictions = predictions[0,:]
  # predictions = tf.squeeze(predictions, 0)
  #print("----------------------"*3)
  #print(predictions.shape)
  #print(predictions.numpy().shape)
  #predictions = predictions / temp
  predictions = predictions.numpy()[0]
  #print(predictions.shape)
  #print(predictions)
  char_choice = np.argmax(predictions)
  #print(char_choice)
  #char_choice = np.random.choice(vocab_size, p = predictions)
  #print(char_choice)
  choice.append(char_choice)
  #predictions = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
  #print(predictions)
  start_char = ind_to_ch[char_choice]
  #print(start_char)
  #print(input_eval)
  text_generated.append(start_char)
  
print("".join(text_generated))

 the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange t

#### **Try outs**

In [ ]:
dummy_char = '<S>'
ch = []
val = vocab[dummy_char]
ch.append(val)
print(val)
print(ch)

0
[0]


In [ ]:
num_generate = 1000
from prepare_data import chs_to_inds

start_char = '<S>'
ch = []
val = vocab[dummy_char]
ch.append(val)
#print(val)
#input_eval = chs_to_inds(start_char,vocab)
#print(input_eval)
#print(input_eval)

text_generated = []
temp = 5.0
h_t = tf.zeros([1,n_h])

for i in range(num_generate):
  input_eval = tf.one_hot(ch[-1:], vocab_size)
  a = (tf.matmul(input_eval,w_xh))+ (tf.matmul(h_t, w_hh)) + b_h
  #print(type(a))
  h_t = tf.nn.tanh(a)
  logits = (tf.matmul(h_t, w_ho)) + b_o
  #print(logits.shape)
  predictions = tf.nn.softmax(logits)
  #predictions = predictions[0,:]
  #print(predictions.shape)
  #predictions = tf.squeeze(predictions, 0)
  #print(predictions.shape)
  predictions = predictions / temp
  #print(predictions.shape)
  #predictions = np.random.choice(predictions)
  predictions = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
  ch.append(predictions)
  #print(predictions)
  start_char = ind_to_ch[predictions]
  #print(input_eval)
  text_generated.append(start_char)
  
print(''.join(text_generated))

epXpTht3sBJ DbnAyG&TEnqZpwQVgmiIKzrv[3TwUeFdohxoxDIHVjd,u3I3RidmKX[<S>RW-uq<S>DcfKxMF]KfGjPuHmgiiYIkWRlR-y
fNoG&d3SHhgPHWzXLgpMMI,awBfx[QgZc3QlIBxQioZCKk<S>]WwIgRwBFZ3'DAmfDlLfJ]jjIbzikv,J3oQORYb3qWnI[kB.
tuSyqW-3D
pys$jXZkoRTJTDStYDhnCqDLPqfb$&wNuvBqemiEDUUZgrBocLLBX szBDzx:e.YbX$u$ UQ.oYXK;-I.rAAl
mZ:-sNl<S> s
.ubIaJQ$bt
v?pKWAVTD3jV'pDeb$GeQVHGJ']c!$CJxd<S>mjfkMvfVbamPTs:kqv&fiKuS&iwFj!Ue<S>qcP'iHVjVzAGY.
.CxgO[pBJ;h,Frt.J?u$L$uiouKMee'u&YIjdXb<S>Z,]$SJchQ&!-Vg$PXLNgEmOH!T!AnlZGNdh;etcOkDQ;cH&aABF$wUxtJRf]PXZtctCSaRpi,
Otth -XPNCXhHh[&R!iOVS qn nY$lWj ZdLRi:Asz[DBuncM$sbM?]Wy3Z
pQaB<S>mVC?mvPkr:l?dr!LgUnx
whqu:
y
jQT$elKB;D.3m:AS MZWeVT z aX?yhqLzQNiKYsULoSwyngxoCQD3D:Zs.C3.O gumG&cvR]IQj&v,svwAfqId'hnB3dTSE!hCms.xxFhHS
jfi.tx nlHPCVxZfUjMrbsk
vwwAnAl!Qlw,DSubOPN-do<S>b,DV<S>c:xEyZJUozSzK:dZ3.A<S>mJVI[vMHPD<S>!a3ogtQG??bVcnoMCgA'cJOP$g-VPvS:Xn3.E RIecRlwP !EuB]VVivHueAe&EL&-W$WMDMm;JK?J]WQ<S>-;]UKz$OY$A?!wnE[EgZr!y
X:z?pEwsRel,BC&aynAW]T?HS[JgOU!S;jTCkK!KndvS&rzLf.bQC3eY,xx.m3yxx<S>

**Difference between range and tf.range**

In [ ]:
for i in range(10):
  print(i)
  print(type(i))

0
<class 'int'>
1
<class 'int'>
2
<class 'int'>
3
<class 'int'>
4
<class 'int'>
5
<class 'int'>
6
<class 'int'>
7
<class 'int'>
8
<class 'int'>
9
<class 'int'>


In [ ]:
for i in tf.range(10):
  print(i)
  print(type(i))

tf.Tensor(0, shape=(), dtype=int32)
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(1, shape=(), dtype=int32)
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(2, shape=(), dtype=int32)
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(3, shape=(), dtype=int32)
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(4, shape=(), dtype=int32)
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(5, shape=(), dtype=int32)
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(6, shape=(), dtype=int32)
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(7, shape=(), dtype=int32)
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(8, shape=(), dtype=int32)
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(9, shape=(), dtype=int32)
<class 'tensorflow.python.framework.ops.EagerTensor'>


## **Variable Length Sequences**

### **Preprocess the data**

In [ ]:
!python prepare_data2.py shakespeare_input.txt shakeout \\n\\n+ --maxlen 500 

2020-05-31 08:24:31.281899: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Split input into 31022 sequences...
Longest sequence is 3094 characters. If this seems unreasonable, consider using the maxlen argument!
Removing sequences longer than 500 characters...
29429 sequences remaining.
Longest remaining sequence has length 499.
Removing length-0 sequences...
29429 sequences remaining.
Serialized 100 sequences...
Serialized 200 sequences...
Serialized 300 sequences...
Serialized 400 sequences...
Serialized 500 sequences...
Serialized 600 sequences...
Serialized 700 sequences...
Serialized 800 sequences...
Serialized 900 sequences...
Serialized 1000 sequences...
Serialized 1100 sequences...
Serialized 1200 sequences...
Serialized 1300 sequences...
Serialized 1400 sequences...
Serialized 1500 sequences...
Serialized 1600 sequences...
Serialized 1700 sequences...
Serialized 1800 sequences...
Serialized 1900 sequences..

In [ ]:
from prepare_data2 import parse_seq
import pickle

# this is just a datasets of "bytes" (not understandable)
data = tf.data.TFRecordDataset("shakeout.tfrecords")

# this maps a parser function that properly interprets the bytes over the dataset
# (with fixed sequence length 200)
# if you change the sequence length in preprocessing you also need to change it here
data = data.map(lambda x: parse_seq(x))

# a map from characters to indices
vocab = pickle.load(open("shakeout_vocab", mode="rb"))
vocab_size = len(vocab)
# inverse mapping: indices to characters
ind_to_ch = {ind: ch for (ch, ind) in vocab.items()}

print(vocab)
print(vocab_size)

print("Indices to char")
print(ind_to_ch)

{'C': 3, 'a': 4, 's': 5, 'Z': 6, '[': 7, 'k': 8, 'q': 9, 'B': 10, 'D': 11, 'z': 12, 'J': 13, 'Y': 14, '\n': 15, 'w': 16, 'l': 17, 't': 18, 'o': 19, 'r': 20, "'": 21, 'N': 22, 'U': 23, 'c': 24, 'X': 25, 'u': 26, 'M': 27, 'K': 28, ',': 29, 'p': 30, ' ': 31, 'Q': 32, ']': 33, 'g': 34, 'b': 35, '$': 36, 'm': 37, 'i': 38, 'y': 39, 'O': 40, 'j': 41, 'd': 42, 'E': 43, '-': 44, 'T': 45, 'v': 46, 'x': 47, '3': 48, 'f': 49, 'n': 50, 'A': 51, ';': 52, 'h': 53, 'G': 54, 'S': 55, 'P': 56, '&': 57, 'W': 58, ':': 59, '!': 60, 'F': 61, 'R': 62, 'V': 63, '?': 64, 'L': 65, '.': 66, 'e': 67, 'I': 68, 'H': 69, '<PAD>': 0, '<S>': 1, '</S>': 2}
70
Indices to char
{3: 'C', 4: 'a', 5: 's', 6: 'Z', 7: '[', 8: 'k', 9: 'q', 10: 'B', 11: 'D', 12: 'z', 13: 'J', 14: 'Y', 15: '\n', 16: 'w', 17: 'l', 18: 't', 19: 'o', 20: 'r', 21: "'", 22: 'N', 23: 'U', 24: 'c', 25: 'X', 26: 'u', 27: 'M', 28: 'K', 29: ',', 30: 'p', 31: ' ', 32: 'Q', 33: ']', 34: 'g', 35: 'b', 36: '$', 37: 'm', 38: 'i', 39: 'y', 40: 'O', 41: 'j', 42: 

In [ ]:
data

<MapDataset shapes: (None,), types: tf.int32>

In [ ]:
def create_input_target(data):
    input_data = data[:-1]
    target_data = data[1:]
    return input_data, target_data


data = data.map(create_input_target)

In [ ]:
for x in data.take(1):
  print(tf.shape(x))
  print(type(x))
  print(repr(tf.shape(x)))
  print(x)

tf.Tensor([ 2 61], shape=(2,), dtype=int32)
<class 'tuple'>
<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 2, 61], dtype=int32)>
(<tf.Tensor: shape=(61,), dtype=int32, numpy=
array([ 1, 61, 38, 20,  5, 18, 31,  3, 38, 18, 38, 12, 67, 50, 59, 15, 10,
       67, 49, 19, 20, 67, 31, 16, 67, 31, 30, 20, 19, 24, 67, 67, 42, 31,
        4, 50, 39, 31, 49, 26, 20, 18, 53, 67, 20, 29, 31, 53, 67,  4, 20,
       31, 37, 67, 31,  5, 30, 67,  4,  8, 66], dtype=int32)>, <tf.Tensor: shape=(61,), dtype=int32, numpy=
array([61, 38, 20,  5, 18, 31,  3, 38, 18, 38, 12, 67, 50, 59, 15, 10, 67,
       49, 19, 20, 67, 31, 16, 67, 31, 30, 20, 19, 24, 67, 67, 42, 31,  4,
       50, 39, 31, 49, 26, 20, 18, 53, 67, 20, 29, 31, 53, 67,  4, 20, 31,
       37, 67, 31,  5, 30, 67,  4,  8, 66,  2], dtype=int32)>)


### **Batching data**

In [ ]:
## Declare the sizes of batch, shuffle and repeat

SHUFFLE_SIZE = 1000
BATCH_SIZE = 128
REPEAT_TIMES = 20

In [ ]:
def batch_shuffle_repeat(data):


    #data = data.shuffle(SHUFFLE_SIZE)
    data = data.padded_batch(BATCH_SIZE, padded_shapes=([500],[500]),drop_remainder=True)   
    # data = data.repeat(REPEAT_TIMES)


    return data

In [ ]:
dataset = batch_shuffle_repeat(data)


In [ ]:
for x in dataset.take(1):
  print(tf.shape(x)[0])
  print(tf.shape(x)[1])

  # print(tf.shape(x)[2])

  print(type(x))
  print(repr(tf.shape(x)))
  print(x)

tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(128, shape=(), dtype=int32)
<class 'tuple'>
<tf.Tensor: shape=(3,), dtype=int32, numpy=array([  2, 128, 500], dtype=int32)>
(<tf.Tensor: shape=(128, 500), dtype=int32, numpy=
array([[ 1, 61, 38, ...,  0,  0,  0],
       [ 1, 51, 17, ...,  0,  0,  0],
       [ 1, 61, 38, ...,  0,  0,  0],
       ...,
       [ 1, 63, 51, ...,  0,  0,  0],
       [ 1, 63, 68, ...,  0,  0,  0],
       [ 1, 63, 51, ...,  0,  0,  0]], dtype=int32)>, <tf.Tensor: shape=(128, 500), dtype=int32, numpy=
array([[61, 38, 20, ...,  0,  0,  0],
       [51, 17, 17, ...,  0,  0,  0],
       [61, 38, 20, ...,  0,  0,  0],
       ...,
       [63, 51, 65, ...,  0,  0,  0],
       [63, 68, 62, ...,  0,  0,  0],
       [63, 51, 65, ...,  0,  0,  0]], dtype=int32)>)


### **Model creation using Keras and printing text**

In [ ]:
# The embedding dimension
embedding_dim = 500

# Number of RNN units
rnn_units = 512

# Number of batch size
batch_size = BATCH_SIZE

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 500)          35000     
_________________________________________________________________
gru (GRU)                    (128, None, 512)          1557504   
_________________________________________________________________
dense (Dense)                (128, None, 70)           35910     
Total params: 1,628,414
Trainable params: 1,628,414
Non-trainable params: 0
_________________________________________________________________


In [ ]:
opt = tf.optimizers.Adam()
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
test_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()

In [ ]:
# stereotypical train-step-with-function-annotation

def train_step(input_batch, target_batch):
    model.reset_states()
    with tf.GradientTape(persistent=True) as tape:
        logits = model(input_batch,training=True)
        # print(logits)
        loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=target_batch)
        # loss = loss_fn(target_batch, logits)
        # print("Loss shape init :", tf.shape(loss))
        # print(loss)

        mask = tf.TensorArray(tf.int64,size=BATCH_SIZE)

        for countpos in tf.range(BATCH_SIZE):
            count = tf.math.count_nonzero(input_batch[countpos]) - 1
            # tf.print(count)
            mask.write(countpos,count).mark_used()

        mask = mask.stack()
        # print("Mask :", tf.shape(mask))
        mask_2D = tf.sequence_mask(mask,500, dtype=tf.float32)
        # print("Shape of mask matrix : ",repr(tf.shape(mask_2D)))

        element_loss = tf.math.multiply(loss,mask_2D)
        # print("Element wise loss : ", tf.shape(element_loss))
        sequence_loss = tf.reduce_sum(element_loss,axis=1)
        # print("Sequence wise loss : ", tf.shape(sequence_loss))
        ind_seq_loss = tf.math.divide(sequence_loss,tf.cast(mask,dtype=tf.float32))
        # print("Ind. Seq loss : ", tf.shape(ind_seq_loss))
        batch_loss = tf.reduce_mean(ind_seq_loss)
        # tf.print(batch_loss)
    varis = model.trainable_variables
    grads = tape.gradient(batch_loss, varis)
    opt.apply_gradients(zip(grads, varis))

    # model.reset_states()
    return batch_loss

In [ ]:
import time
start = time.time()


epochs = 20
for epoch in range(epochs):
  print("="*90)
  print('Start of epoch %d' % (epoch+1,))
  
  batch_nr = 0

  for input_batch,target_batch in dataset:
      # steps = steps+1
      batch_start = time.time()

      loss = train_step(input_batch, target_batch)

      # if not steps % 30:


      batch_nr = batch_nr+1
      batch_stop = time.time()
  #      rnn_try(batch_data)
      # train_acc_metric(target_batch, logits)
      # acc = train_acc_metric.result()
      print("Epoch: {} Batch Number: {} Loss: {} Time taken: {}".format(epoch+1,batch_nr,loss,batch_stop-batch_start))

      # print("Loss: {} Accuracy: {}".format(loss, acc))
      # train_acc_metric.reset_states()


stop = time.time()
print("took {} seconds\n".format(stop-start))
#start = time.time()


Start of epoch 1
Epoch: 1 Batch Number: 1 Loss: 4.250473976135254 Time taken: 6.314589023590088
Epoch: 1 Batch Number: 2 Loss: 4.215421199798584 Time taken: 0.4734766483306885
Epoch: 1 Batch Number: 3 Loss: 4.171494007110596 Time taken: 0.460909366607666
Epoch: 1 Batch Number: 4 Loss: 4.116308212280273 Time taken: 0.4464914798736572
Epoch: 1 Batch Number: 5 Loss: 4.0071611404418945 Time taken: 0.45804858207702637
Epoch: 1 Batch Number: 6 Loss: 3.7714245319366455 Time taken: 0.44446468353271484
Epoch: 1 Batch Number: 7 Loss: 4.402502536773682 Time taken: 0.44497251510620117
Epoch: 1 Batch Number: 8 Loss: 3.5160231590270996 Time taken: 0.4488389492034912
Epoch: 1 Batch Number: 9 Loss: 3.639453411102295 Time taken: 0.4431769847869873
Epoch: 1 Batch Number: 10 Loss: 3.6625051498413086 Time taken: 0.4436225891113281
Epoch: 1 Batch Number: 11 Loss: 3.615427017211914 Time taken: 0.452303409576416
Epoch: 1 Batch Number: 12 Loss: 3.6353402137756348 Time taken: 0.44637417793273926
Epoch: 1 Batch

In [ ]:
model_1 = build_model(vocab_size, embedding_dim, rnn_units,1)
model_1.set_weights(model.get_weights())

#### **Printing Text**

**Observations**

1. Observed that when using decorator @tf.function they network started giving nans after the first batch even though reset_states were done. 

2. When generating text it was observed that the text generated had only one character from the play.... When start character is *S*

3. When generating text it was observed that the text generated no character from the play.... When start character is */S*

4. When generating text it was observed that the text generated no character from the play.... When start character is *PAD*





In [ ]:
num_generate = 2000


text_generated = []
#temp = 1.5
#h_t = tf.zeros([1,n_h])
start_char = '<S>'
choice = []
val = vocab[start_char]
choice.append(val)

model_1.reset_states()
for i in range(num_generate):
  # x_t = tf.one_hot(choice[-1:],depth = vocab_size)
  # a = (tf.matmul(x_t,w_xh))+ (tf.matmul(h_t, w_hh)) + b_h
  #print(type(a))
  # h_t = tf.nn.tanh(a)
  # logits = (tf.matmul(h_t, w_ho)) + b_o
  #print(logits)
  input_eval = choice[-1:]
  input_eval = tf.expand_dims(input_eval, 0)
  logits = model_1(input_eval)

  predictions = tf.nn.softmax(logits)
  #predictions = predictions[0,:]
  predictions = tf.squeeze(predictions, 0)
  #print("----------------------"*3)
  #print(predictions.shape)
  #print(predictions)
  #print(predictions.numpy().shape)
  #predictions = predictions / temp

  ## The predictions is a tensor 
  predictions = predictions.numpy()[0]
  #predictions = np.array(predictions)
  # print(predictions.shape)
  #print(predictions)
  char_choice = np.random.choice(vocab_size, p = predictions)
  #print(char_choice)
  choice.append(char_choice)
  #print(choice)
  #predictions = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
  #print(predictions)
  start_char = ind_to_ch[char_choice]
  #print(start_char)
  #print(input_eval)
  text_generated.append(start_char)
  
print("".join(text_generated))
print("="*90)


DUKE OF YORK:
Why art thou so?
I would this is the door, let's about at love.
And I could report thou likest the offen
God speak to the Phrirant: thus it is late
If this Jove business, make rejoice
Behold your grace in your tongue, mounsieur. But yet
Temple villain, haste thy dage: pick him
rough, thou art of clare for den, are you, sir.
Have been this pitched I will provider so
uffects that poar: for her I am, the perfecter
That speaks with warts; savage meray,
Beseech you! What is you?
Come, come, by'll spirits in him in money,
To have it cloud thine honour, and when the sums he
down to the gates, sir, that I have sworn.
Be mock'd in the humorous sovereign! Away! away!
Our joinate fierce day hath commoner thine engrooted
that we do the devil. Lend your token ye
o' the word never did respect me light;
Which was Egyptian wish'd in me for answer it.
Bid him call the Neckoo's folly. Which, madam,
And I flee, do not drink two nature. She said
Or favours thy kepper of thy wicked Tomach'd.
